# Medical Transcriptions Categorization
Freya Gray
CS39AA - Natural Language Processing
<br>
This project uses medical transcripts to try and classify the medical specialty that the transcript originated from. The dataset is from the Medical Transcriptions dataset on [Kaggle](https://www.kaggle.com/tboyle10/medicaltranscriptions). 
This notebook will implement a baseline model for the classification problem

## Imports

In [ ]:
import numpy as np 
import pandas as pd 
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk import WordNetLemmatizer
from nltk.corpus import stopwords

nltk.download('stopwords')

### Import Dataset

In [ ]:
df = pd.read_csv('mtsamples.csv')
df.head()

## Clean Dataset

Remove nonessential columns and drop null entries

In [ ]:
df.drop(['Unnamed: 0','description','sample_name','keywords'], axis = 1, inplace = True)
df.dropna(inplace = True)
df.reset_index(drop = True, inplace = True)
df.describe()

Convert transcripts to lowercase, lemmatize and remove stopwords

In [ ]:
stop_words = stopwords.words("english")
tokeniser = RegexpTokenizer("[A-Za-z]+")
tokens = tokeniser.tokenize(df['transcription'])
lemmatiser = WordNetLemmatizer()
df['transcription_clean'] = [lemmatiser.lemmatize(t.lower(), "v") for t in tokens]
df['transcription_clean'] = [t for t in df['transcription'] if t not in stop_words]
df.head()

## Create Train and Test Sets

## Label Encoding
Convert targets to int

## Model 